In [3]:
# Installation is necessary because the Colab environment is reset every time.
# We install the official Ultralytics library which provides the YOLO models.
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
"""
Trains the YOLOv8n model using Transfer Learning on the custom dataset.

The YOLOv8n (nano) model is chosen for its speed and small size, which is ideal
for training on a small dataset (77 images) and mitigating the risk of
overfitting compared to larger models.

The training parameters are set as follows :
- DATA_PATH : Specifies the path to our custom data.yaml file, which tells YOLO
  where to find the train/valid/test images and our three class names.
- epochs = 50 : The model runs through the entire dataset 50 times.
- imgsz = 640 : The input images are resized to 640x640 pixels, which is the
  standard input size for YOLOv8.
"""
# Example path (mine) :
DATA_PATH = '/content/drive/MyDrive/Labelling images.v2i.yolov8/data.yaml'

from ultralytics import YOLO

# 1). Load the pre-trained nano model. This is the base model that already knows
# general object features (pre-trained on COCO).
model = YOLO('yolov8n.pt')

# 2). Launch the training process (Fine-Tuning)
# This process takes the knowledge from 'yolov8n.pt' and specializes it
# to detect my bottle of water, my headphones, and my pencil case.
model.train(data=DATA_PATH, epochs=50, imgsz=640)

Ultralytics 8.3.231 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Labelling images.v2i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f13b1069340>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

The Transfer Learning process successfully adapted the pre-trained YOLOv8 Nano model to the highly specific task of detecting personal objects.
Training logs show a rapid convergence, starting with a low baseline mAP and achieving a 96.7% mAP 0.5 on the validation set by Epoch 50.

Performance across classes was near-perfect, notably achieving 99.5% mAP 0.5 for the 'bottle of water' and 'pencil case' classes, while the 'headphones' class, due to its more complex and varied shapes in the dataset, still scored a very high 91.2%.

This result confirms the efficacy of using a small, quality-controlled dataset for fine-tuning in a constrained domain, providing strong empirical evidence of the project's primary contribution.

In [6]:
"""
Downloads the complete model training results from the Colab environment.

Because we trained the model in the cloud (Colab) to avoid 'Segmentation Fault'
issues on the local machine, this step is necessary to retrieve the essential
files (the trained model and the evaluation graphs) to continue the project locally.

How it works:
1). Compresses the results folder ('train2') into a single ZIP file.
2). Triggers the download dialogue in the web browser.
"""
from google.colab import files
import shutil
import os

# 1). Specify the path to the results folder
# This is the folder generated by the training process, containing best.pt and the plots.
folder_to_download = "/content/runs/detect/train2"
zip_filename = "train2_results.zip"

# 2). Create the ZIP file from the folder
# 'shutil.make_archive' is used to compress the folder because Colab cannot
# download entire folders directly.
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', folder_to_download)

# 3. Trigger the browser download
# This Colab API command initiates the Windows/Mac download window.
files.download(zip_filename)

print("Téléchargement du dossier 'train2' en cours...")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Téléchargement du dossier 'train2' en cours...


In [2]:
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Model valuation cell
"""
Performs the crucial final comparison of both the Baseline and Custom models
on the independent Test Set.

This step is mandatory for academic rigor (Crit. B) and directly calculates the
Key Performance Indicator (KPI) used in the final report: the mAP@0.5 score.
The huge difference in performance between the two models (Custom vs. Baseline)
justifies the entire project's contribution.

The 'val' (validation) mode calculates the standard metrics, ensuring the
performance figures are accurate and reproducible.
"""
from ultralytics import YOLO

# This variable must point to the data.yaml file, which links to the dataset splits.
DATA_PATH = '/content/drive/MyDrive/Labelling images.v2i.yolov8/data.yaml'

# A). Evaluation du modèle baseline

print("\n--- A. Évaluation du Modèle Baseline (YOLOv8n) ---")
# Load the standard pre-trained model (trained on COCO, not our objects).
baseline_model = YOLO('yolov8n.pt')

# Run formal validation on the TEST set. split='test' guarantees the model has
# never seen these images. plots=False keeps the output clean for the baseline.
metrics_baseline = baseline_model.val(data=DATA_PATH, split='test', plots=False)

# Extract the mAP50 score for the final report table.
baseline_mAP50 = metrics_baseline.results_dict['metrics/mAP50(B)']
print(f"Baseline mAP50 on Test Set: {baseline_mAP50:.4f}")


# B). Custom Model Evaluation (The Project Contribution)

# Path to the custom model weights from the training session.
CUSTOM_MODEL_PATH = '/content/drive/MyDrive/Labelling images.v2i.yolov8/weights/best.pt'
print("\n--- B. Évaluation de Votre Modèle Custom ---")

# Load our final, best-performing model
custom_model = YOLO(CUSTOM_MODEL_PATH)

# Run formal validation on the TEST set. plots=True is used here to generate
# the Precision-Recall curve and Confusion Matrix needed for the report documentation.
metrics_custom = custom_model.val(data=DATA_PATH, split='test', plots=True)

# Extract the mAP50 score for the final report table.
custom_mAP50 = metrics_custom.results_dict['metrics/mAP50(B)']
print(f"Custom Model mAP50 on Test Set: {custom_mAP50:.4f}")

# The generated plots are saved in /content/runs/detect/val for later download.

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

--- A. Évaluation du Modèle Baseline (YOLOv8n) ---
Ultralytics 8.3.231 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.1 ms, read: 2.7±0.9 MB/s, size: 1174.7 KB)
val: Scanning /content/drive/MyDrive/Labelling images.v2i.yolov8/test/labels... 8 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 8/8 1.2it/s 6.7s
val: New cache created: /content/drive/MyDrive/Labelling images.v2i.yolov8/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 3.8s/it 3.8s
                   

The side-by-side evaluation on the unseen Test Set (8 images) confirms the success of the custom training process.

The Baseline YOLOv8n Model scored an extremely low 2.65% mAP 0.5, proving its inability to detect the custom objects without specific fine-tuning.

In contrast, the Custom-Trained Model achieved an outstanding 99.50% mAP 0.5 and a strong 79.6% mAP 0.5:0.95, indicating near-perfect accuracy and precise localization.

Notably, the model achieved 99.5% mAP for all three classes individually, conclusively demonstrating that Transfer Learning successfully specialized the YOLO architecture to the specific constraints of the custom dataset, thereby fulfilling the core project objective.